In [71]:
import csv
import pandas as pd

# install libraries the first time you are using this notebook, 
# you can skip the installation process later by commenting the following three lines:
!pip install "sdv==0.18.0" 
!pip install "sdmetrics==0.9.1"

from sdv.metrics.tabular import KSComplement
from sdv.lite import TabularPreset
from sdv.evaluation import evaluate
from dataclasses import dataclass

In [77]:
# This Notebook uses a config file to create interpolated data from an input data set. 
# The interpolation is customizable by config params. See readme.md for more details.

config = pd.read_json("config.json") #config file(dictionary)
data_file = config["Input File"][0]

target_file = config["Output File"][0]
data = pd.read_csv(str(data_file), sep=config["Input Column Separator"][0])
headers = data.columns

feature_dict = config["Features"]

features = [] #names of relevant headers
for feature in config["Features"].keys():
    features.append(feature) # wanted headers

allowed_percentage = config["Percentage"][0]
number_of_samples = int(config["N_samples"][0])
#print(features)

print("Initialization:\nconfig file read, input is taken from file with name '",data_file, "'")

Initialization:
config file read, input is taken from file with name ' combined.tsv '


In [78]:
#Remove unspecified columns
for header in headers:
    if not header in features:
        data.pop(header)
        #print("Deleting column", header, "because of configuration.")

#Remove columns with to many NAs (missing values)
headers = data.columns #fixed list of columns we need
for header in headers:
    na_percentage = data[header].isna().sum() / len(data[header].index)
    #print("NA in % for", header, ":", na_percentage)
    if na_percentage > allowed_percentage:
        data.pop(header)
        headers = headers.drop(header)
        print("Deleting column", header, "because of" ,"{:.2%}".format(na_percentage), "NAs.")

#removing unwanted NAs and interpolating others
numericals = []
for x in headers:
    #print(feature_dict, feature_dict[x][0])
    if feature_dict[x][0] == "numerical":
        numericals.append(x)
        
print("configured columns:",len(features))
print("number of columns after removing columns that have too many missing values (NAs) or cannot be resolved:", len(headers))
print("how many columns have numericals:", len(numericals))

for numerical in numericals:
    data[numerical] = pd.to_numeric(data[numerical], errors='coerce')

data = data.interpolate(method='pad')

Deleting column Neoplasm Histologic Grade because of 76.64% NAs.
Deleting column Treatment Details before PDX because of 94.74% NAs.
configured columns: 18
number of columns after removing columns that have too many missing values (NAs) or cannot be resolved: 16
how many columns have numericals: 3


In [79]:
#create validation set

validation_set = data.sample(frac=0.3)
#removal = data.sample(frac=0.9)
#data = data.drop(removal.index)
validation_set.reset_index()

print("rows in original data:", len(data))
print("rows for validation:", len(validation_set))

rows in original data: 1541
rows for validation: 462


In [80]:
#Generating Metadata for SV
metadata = {}
metadata["fields"] = {}

for x in headers:
    metadata["fields"][x] = {}
    metadata["fields"][x]["type"] = feature_dict[x][0]
    if feature_dict[x][0] == "numerical":
        metadata["fields"][x]["subtype"] = feature_dict[x][1]
metadata["constraints"] = []

print(metadata)

{'fields': {'Cancer Type Detailed': {'type': 'categorical'}, 'Mutation Count': {'type': 'numerical', 'subtype': 'integer'}, 'Oncotree Code': {'type': 'categorical'}, 'Overall Survival (Months)': {'type': 'numerical', 'subtype': 'float'}, 'Sample Type': {'type': 'categorical'}, 'Diagnosis Age': {'type': 'numerical', 'subtype': 'integer'}, 'TERT': {'type': 'categorical'}, 'IDH1': {'type': 'categorical'}, 'PABPC3': {'type': 'categorical'}, 'PTEN': {'type': 'categorical'}, 'SPRY3': {'type': 'categorical'}, 'MUC12': {'type': 'categorical'}, 'GXYLT1': {'type': 'categorical'}, 'SMARCA4': {'type': 'categorical'}, 'FAT1': {'type': 'categorical'}, 'MUC5B': {'type': 'categorical'}}, 'constraints': []}


In [81]:
#Method to fit model to the data and generate data with SDV's TabularPreset
def makeData(x):
    model = TabularPreset(name='FAST_ML', metadata=metadata) #SDV's FAST_ML preset uses ML to model your data
    model.fit(data)

    new_data = model.sample(num_rows=number_of_samples)
    new_data = new_data.round(decimals = 2)
    new_data.to_csv(target_file, index=False)
    print(x+1, ".: Writing to", str(target_file), "The generated data is", 
          "{:.2%}".format(evaluate(new_data, validation_set, metrics=['KSComplement'])), 
          "accurate to the original data.",
          "(if unsatisfactory try another config)")# using less or diff. features or more data or removing more NAs.")
    
    return "{:.2%}".format(evaluate(new_data, validation_set, metrics=['KSComplement']))


# create the new interpolated data with dynamic file name depending on config params
naInPercent = (allowed_percentage*100).astype(int)
nrOfFeatures = len(features)
nrOfRows = len(data)
dataframe = []
filename = str(naInPercent)+str(data_file.replace('.', ''))+"_%NaNs_Interpolated_"+str(nrOfFeatures)+"Features_"+str(nrOfRows)+"InputLines.csv"

print("Making data in iterations:\n")
for x in range(0,10):
    dataframe.append(makeData(x))

print("\nAccuracies from", x+1, "generated results written to", filename, "for statistical analysis.")
print("\nThe actual newly generated output with synthetically generated data has been successfully written to "+str(target_file)+".")
print("Use it to hand it over to someone else or tweak your synthetic data by achanging the config.")

file = open(filename, "w", newline="")

writer = csv.writer(file)

for val in dataframe:
    writer.writerow([val])

file.close()

Making data in iterations:

1 .: Writing to output.csv The generated data is 77.30% accurate to the original data. (if unsatisfactory try another config)
2 .: Writing to output.csv The generated data is 77.23% accurate to the original data. (if unsatisfactory try another config)
3 .: Writing to output.csv The generated data is 77.22% accurate to the original data. (if unsatisfactory try another config)
4 .: Writing to output.csv The generated data is 77.13% accurate to the original data. (if unsatisfactory try another config)
5 .: Writing to output.csv The generated data is 77.60% accurate to the original data. (if unsatisfactory try another config)
6 .: Writing to output.csv The generated data is 76.52% accurate to the original data. (if unsatisfactory try another config)
7 .: Writing to output.csv The generated data is 77.85% accurate to the original data. (if unsatisfactory try another config)
8 .: Writing to output.csv The generated data is 77.93% accurate to the original data. (if